Goal: Build an agent that writes a tweet for you.

The Writer: drafts a tweet based on a topic.

The Critic: checks if it is too long (> 280 chars) or "boring".

The Loop: If bad, send it back to the Writer.

The Gatekeeper (HiTL): If the Critic approves, PAUSE. Wait for your manual approval before "posting" (printing) it.

In [ ]:
import operator
from typing import Annotated, TypedDict, List, Union, Dict, Any
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver


In [ ]:
#define state

class TweetState(TypedDict):
    topic: str
    draft: str | None =  None
    critique: str | None = None
    revision_count: int = 0



In [ ]:
#define nodes

def generate_tweet(state: TweetState) -> Dict[str, Any]:
    print("Writer: Genrating Draft")
    topic = state['topic']
    count = state['revision_count']

    #mock logic 
    if count == 0:
        draft = f"Draft for topic: {topic}"
    else:
        draft = f"Draft for topic: {topic} (Revision {count})"
    return {
        "draft": draft,
        "revision_count": count +1
    }


def critique_tweet(state: TweetState) -> Dict[str, Any]:
    print("Critique: Reviewing Draft")
    draft = state['draft']
    if len(draft)>100:
        critique = "Too Long"
    else:
        critique = "Acceptable"
        
    return {"critique": critique}

def publish_tweet(state: TweetState) -> Dict[str, Any]:
    print("Publisher: Publishing Tweet")
    draft = state['draft']
    return {"published_tweet": draft}

In [ ]:
#build graph
builder = StateGraph(TweetState)
#define node
builder.add_node("writer", generate_tweet)
builder.add_node("critique", critique_tweet)
builder.add_node("publisher", publish_tweet)

#add edge
builder.add_edge(START, "writer")
builder.add_edge("writer", "critique")

#add conditional logic
def decide_next_step(state: TweetState):
    if state['revision_count'] == 3:
        print(f'Maximum revision count reached: {state["revision_count"]}')
        return "publisher"
    if state['critique'] == "Acceptable":
        return "publisher"
    else:
        return "writer"

builder.add_conditional_edges("critique", decide_next_step, {
    "publisher": "publisher",
    "writer": "writer"
})


#memory store
memory = MemorySaver()

#compile
app = builder.compile(checkpointer=memory,
interrupt_before = ['publisher'])


In [ ]:

thread = {
    "configurable":{
        "thread_id" : "tweet_test_1",
    }
}
msg = TweetState({"topic": "Langraph Langraph Langraph Langraphraph L", 
"revision_count":0})
for event in app.stream(msg, thread):
    print(event)
    pass
print("Pause for human")
current_state = app.get_state(thread)
print(f'Pending state: {current_state}')
user_input = input("Type 'ok' to publish: ")

if user_input == "ok":
    for event in app.stream(msg, thread):
        print(event)
    

Writer: Genrating Draft
{'writer': {'draft': 'Draft for topic: Langraph Langraph Langraph Langraphraph L', 'revision_count': 1}}
Critique: Reviewing Draft
{'critique': {'critique': 'Acceptable'}}
{'__interrupt__': ()}
Pause for human
Pending state: StateSnapshot(values={'topic': 'Langraph Langraph Langraph Langraphraph L', 'draft': 'Draft for topic: Langraph Langraph Langraph Langraphraph L', 'critique': 'Acceptable', 'revision_count': 1}, next=('publisher',), config={'configurable': {'thread_id': 'tweet_test_1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e3f7e-6747-6b4c-801a-c8a08af26633'}}, metadata={'source': 'loop', 'step': 26, 'parents': {}}, created_at='2025-12-28T14:17:11.130600+00:00', parent_config={'configurable': {'thread_id': 'tweet_test_1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e3f7e-6745-6c84-8019-8151781127f7'}}, tasks=(PregelTask(id='c153baa7-7dc9-de6f-0411-824e82927718', name='publisher', path=('__pregel_pull', 'publisher'), error=None, interrupts=(), state=None, re

In [98]:
#define nodes
#define state
import operator
from typing import Annotated, TypedDict, List, Union, Dict, Any
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver

class TweetState(TypedDict):
    topic: str
    draft: str | None =  None
    critique: str | None = None
    revision_count: int = 0


def generate_tweet(state: TweetState) -> Dict[str, Any]:
    print("Writer: Genrating Draft")
    topic = state['topic']
    count = state['revision_count']

    #mock logic 
    if count == 0:
        draft = f"Draft for topic: {topic}"
    else:
        draft = f"Draft for topic: {topic} (Revision {count})"
    return {
        "draft": draft,
        "revision_count": count +1
    }


def critique_tweet(state: TweetState) -> Dict[str, Any]:
    print("Critique: Reviewing Draft")
    draft = state['draft']
    if len(draft)>100:
        critique = "Too Long"
    else:
        critique = "Acceptable"
        
    return {"critique": critique}

def publish_tweet(state: TweetState) -> Dict[str, Any]:
    print("Publisher: Publishing Tweet")
    draft = state['draft']
    return {"published_tweet": draft}#build graph
builder = StateGraph(TweetState)
#define node
builder.add_node("writer", generate_tweet)
builder.add_node("critique", critique_tweet)
builder.add_node("publisher", publish_tweet)

#add edge
builder.add_edge(START, "writer")
builder.add_edge("writer", "critique")

#add conditional logic
def decide_next_step(state: TweetState):
    if state['revision_count'] == 3:
        print(f'Maximum revision count reached: {state["revision_count"]}')
        return "publisher"
    if state['critique'] == "Acceptable":
        return "publisher"
    else:
        return "writer"

builder.add_conditional_edges("critique", decide_next_step, {
    "publisher": "publisher",
    "writer": "writer"
})


#memory store
memory = MemorySaver()

#compile
app = builder.compile(checkpointer=memory,
interrupt_before = ['publisher'])

thread = {
    "configurable":{
        "thread_id" : "tweet_test_1",
    }
}
msg = TweetState({"topic": "Langraph Langraph Langraph Langraphraph L", 
"revision_count":0})
for event in app.stream(msg, thread):
    print(event)
    pass
print("Pause for human")
current_state = app.get_state(thread)
print(f'Pending state: {current_state}')
user_input = input("Type 'ok' to publish: ")

if user_input == "ok":
    for event in app.stream(None, thread):
        print(event)
    

Writer: Genrating Draft
{'writer': {'draft': 'Draft for topic: Langraph Langraph Langraph Langraphraph L', 'revision_count': 1}}
Critique: Reviewing Draft
{'critique': {'critique': 'Acceptable'}}
{'__interrupt__': ()}
Pause for human
Pending state: StateSnapshot(values={'topic': 'Langraph Langraph Langraph Langraphraph L', 'draft': 'Draft for topic: Langraph Langraph Langraph Langraphraph L', 'critique': 'Acceptable', 'revision_count': 1}, next=('publisher',), config={'configurable': {'thread_id': 'tweet_test_1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e3f81-b05c-6618-8002-1d58e7b9ef3c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-28T14:18:39.324310+00:00', parent_config={'configurable': {'thread_id': 'tweet_test_1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e3f81-b05b-61fa-8001-99e1e1f4e5b3'}}, tasks=(PregelTask(id='a7fe7f19-d35f-8d5a-7e02-2eca3095cc8d', name='publisher', path=('__pregel_pull', 'publisher'), error=None, interrupts=(), state=None, res

In [99]:
#reject with max
thread = {
    "configurable":{
        "thread_id" : "tweet_test_1",
    }
}
msg = TweetState({"topic": "Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL", 
"revision_count":0})
for event in app.stream(msg, thread):
    print(event)
    pass
print("Pause for human")
current_state = app.get_state(thread)
print(f'Pending state: {current_state}')
user_input = input("Type 'ok' to publish: ")

if user_input == "ok":
    for event in app.stream(None, thread):
        print(event)

Writer: Genrating Draft
{'writer': {'draft': 'Draft for topic: Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL', 'revision_count': 1}}
Critique: Reviewing Draft
{'critique': {'critique': 'Too Long'}}
Writer: Genrating Draft
{'writer': {'draft': 'Draft for topic: Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL (Revision 1)', 'revision_count': 2}}
Critique: Reviewing Draft
{'critique': {'critique': 'Too Long'}}
Writer: Genrating Draft
{'writer': {'draft': 'Draft for topic: Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL Langraph Langraph Langraph Langraphraph Langraph Langraph Langraph Langraphraph LL (Revision 2)', 'revision_count': 3}}
Critique: Reviewing Draft
Maximum revision count reached: 3
{'critique': {'critique':